In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import os
from sklearn.preprocessing import LabelEncoder 
from imblearn.ensemble import BalancedRandomForestClassifier


import warnings
warnings.filterwarnings('ignore')

In [2]:
le = LabelEncoder()

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier

## Read in Data and preform basic cleaning 


In [4]:
file_path = Path("Resources/world_2019_data.csv")

In [5]:
df = pd.read_csv(file_path)
df

,country_name,country_code,h_2019,edu_2019,e_2019,con_2019,inf_2019,une_2019,p_2019,g_2019,income_group
0,Angola,AGO,2.53,2.923862e+09,2.827171e+10,4.057062e+10,17.08,7.42,31825299,-0.70,Low income
1,Armenia,ARM,11.34,3.161166e+08,5.631686e+09,1.307507e+10,1.44,18.30,2957728,7.60,High income
2,Australia,AUS,9.91,6.777152e+10,3.364310e+11,1.033080e+12,1.61,5.16,25365745,2.11,High income
3,Austria,AUT,10.43,2.241872e+10,2.467240e+11,3.160610e+11,1.53,4.49,8879920,1.49,High income
4,Azerbaijan,AZE,4.04,1.178561e+09,2.362976e+10,3.306206e+10,2.61,4.85,10024283,2.50,High income
...,...,...,...,...,...,...,...,...,...,...,...
123,Ukraine,UKR,7.10,7.662706e+09,6.344856e+10,1.433540e+11,7.89,8.19,44386203,3.20,Low income
124,Uruguay,URY,9.35,2.774959e+09,1.699176e+10,4.865871e+10,7.88,8.88,3461731,0.35,High income
125,United States,USA,16.77,9.540000e+11,2.519730e+12,1.740260e+13,1.81,3.67,328329953,2.29,High income
126,Vanuatu,VUT,3.36,4.506711e+07,4.714967e+08,7.126577e+08,2.76,1.80,299882,3.24,Low income


In [6]:
# #drop string columns
# df = df.drop(columns=["country_name", "country_code"])
# df

In [7]:
#check data types
df.dtypes

country_name     object
country_code     object
h_2019          float64
edu_2019        float64
e_2019          float64
con_2019        float64
inf_2019        float64
une_2019        float64
p_2019            int64
g_2019          float64
income_group     object
dtype: object

In [8]:
#encode y column
df['income_group'] = le.fit_transform(df['income_group'])

In [9]:
#define Varibles
X = df.drop(columns=["country_name", "country_code"])
# X

y = df["income_group"]
y

0      1
1      0
2      0
3      0
4      0
      ..
123    1
124    0
125    0
126    1
127    0
Name: income_group, Length: 128, dtype: int32

In [10]:
y = df["income_group"]
y

0      1
1      0
2      0
3      0
4      0
      ..
123    1
124    0
125    0
126    1
127    0
Name: income_group, Length: 128, dtype: int32

In [11]:
# Check the balance of our target values
y.value_counts()

0    79
1    49
Name: income_group, dtype: int64

In [12]:
df

,country_name,country_code,h_2019,edu_2019,e_2019,con_2019,inf_2019,une_2019,p_2019,g_2019,income_group
0,Angola,AGO,2.53,2.923862e+09,2.827171e+10,4.057062e+10,17.08,7.42,31825299,-0.70,1
1,Armenia,ARM,11.34,3.161166e+08,5.631686e+09,1.307507e+10,1.44,18.30,2957728,7.60,0
2,Australia,AUS,9.91,6.777152e+10,3.364310e+11,1.033080e+12,1.61,5.16,25365745,2.11,0
3,Austria,AUT,10.43,2.241872e+10,2.467240e+11,3.160610e+11,1.53,4.49,8879920,1.49,0
4,Azerbaijan,AZE,4.04,1.178561e+09,2.362976e+10,3.306206e+10,2.61,4.85,10024283,2.50,0
...,...,...,...,...,...,...,...,...,...,...,...
123,Ukraine,UKR,7.10,7.662706e+09,6.344856e+10,1.433540e+11,7.89,8.19,44386203,3.20,1
124,Uruguay,URY,9.35,2.774959e+09,1.699176e+10,4.865871e+10,7.88,8.88,3461731,0.35,0
125,United States,USA,16.77,9.540000e+11,2.519730e+12,1.740260e+13,1.81,3.67,328329953,2.29,0
126,Vanuatu,VUT,3.36,4.506711e+07,4.714967e+08,7.126577e+08,2.76,1.80,299882,3.24,1


In [13]:
y_percent = (49/(79+49))*100
print(f"y = {y_percent}% of training group")

y = 38.28125% of training group


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [15]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
balanced_forest = BalancedRandomForestClassifier(n_estimators=100, random_state=1)


In [16]:
#fit the model
balanced_forest = balanced_forest.fit(X_train, y_train)

In [17]:
#calculate balanced accuracy score
y_pred = balanced_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)


1.0

In [18]:
#display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[20,  0],
       [ 0, 12]], dtype=int64)

In [19]:
cm_df = pd.DataFrame(cm)
cm_df

,0,1
0,20,0
1,0,12


In [20]:
cm_df = cm_df.set_axis(["High Income", "Low Income"], axis = 1)
cm_df = cm_df.set_axis(["High Income", "Low Income"], axis = 0)
cm_df

,High Income,Low Income
High Income,20,0
Low Income,0,12


In [21]:
#Print imbalanced classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        12

    accuracy                           1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32

